In [155]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

In [ ]:
df = pd.read_csv('data_2genre.csv')
df.iloc[:, -1] = df.iloc[:,-1].replace({2:0})

In [ ]:
X = df.iloc[:, 1:-1]
X = normalize(X)
y = df.iloc[:, -1] 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0) 
X_train, X_test, y_train, y_test = X_train.T, X_test.T, y_train.values.reshape(1, 150), y_test.values.reshape(1, 50)

In [ ]:
learning_rate = 0.5
num_iter = 10000

In [156]:
def sigmoid(z):
    return (1 / (1 + np.exp(-z)))

In [ ]:
def train(X_train, y_train, num_iter=10000, learning_rate=0.5, hidden_layer_num=15):
    
    n = X_train.shape[0]
    m = X_train.shape[1]
    w1 = np.random.randn(hidden_layer_num, n) * 0.01 #
    b1 = np.random.randn(hidden_layer_num, 1)
    w2 = np.random.randn(1, hidden_layer_num) * 0.01 #
    b2 = np.random.randn(1, 1)
    for i in range(num_iter):
        Z1 = np.dot(w1, X_train) + b1
        A1 = sigmoid(Z1) 
        Z2 = np.dot(w2, A1) + b2
        A2 = sigmoid(Z2) 
        cost = np.sum((y_train * np.log(A2)) + ((1-y_train) * np.log(1-A2))) * (-1/m)

        dZ2 = A2 - y_train
        dW2 = np.dot(dZ2, A1.T) / m  #######
        db2 = np.sum(dZ2, axis=1, keepdims=True) * (1 / m)

        dA1 = np.dot(w2.T, dZ2)
        dZ1 = dA1 * sigmoid(Z1) * (1 - sigmoid(Z1))
        dW1 = np.dot(dZ1, X_train.T) / m #######
        db1 = np.sum(dZ1, axis=1, keepdims=True) * (1 / m)

        w2 = w2 - learning_rate * dW2
        b2 = b2 - learning_rate * db2
        w1 = w1 - learning_rate * dW1
        b1 = b1 - learning_rate * db1

        if(i %1000 == 0):
            print(f'At epoch {i}, cost is {cost}')
    return [w1, w2, b1, b2]

In [ ]:
params = train(X_train, y_train,num_iter=20000, learning_rate=1, hidden_layer_num=17)

In [ ]:
def score(X, y, w1, w2, b1, b2):
    Z1 = np.dot(w1, X) + b1
    A1 = sigmoid(Z1) 
    Z2 = np.dot(w2, A1) + b2
    A2 = sigmoid(Z2)
    Y_prediction = (A2 > 0.5).astype(float)
    print("Accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction - y)) * 100))
    

In [ ]:
score(X_train, y_train, *params)
score(X_test, y_test, *params)

# Multiclass 

In [220]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

In [158]:
data = pd.read_csv('data.csv')

In [209]:
X = data.iloc[:, 1:-1]
X = normalize(X)

y = data.iloc[:, -1] 
y_labelled = LabelEncoder().fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_labelled, random_state = 0) 
X_train, X_test, y_train, y_test = X_train.T, X_test.T, y_train.reshape(1, 750), y_test.reshape(1, 250)


yone_train = OneHotEncoder(sparse=False).fit_transform(y_train.reshape(-1, 1), y=None)
yone_train = yone_train.T

yone_test = OneHotEncoder(sparse=False).fit_transform(y_test.reshape(-1, 1), y=None)
yone_test = yone_test.T

In [210]:
def train_multi(X_train, y_train, num_iter=10000, learning_rate=0.5, hidden_layer_num=15):
    
    n = X_train.shape[0]
    m = y_train.shape[1]
    w1 = np.random.randn(hidden_layer_num, n) * 0.01 #
    b1 = np.random.randn(hidden_layer_num, 1)
    w2 = np.random.randn(10, hidden_layer_num) * 0.01 #
    b2 = np.random.randn(10, 1)
    for i in range(num_iter):
        Z1 = np.dot(w1, X_train) + b1
        A1 = sigmoid(Z1) 
        Z2 = np.dot(w2, A1) + b2
        A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0) # SOFTMAX
        cost = np.sum(y_train * np.log(A2)) * (-1/m)

        dZ2 = A2 - y_train
        dW2 = np.dot(dZ2, A1.T) / m  #######
        db2 = np.sum(dZ2, axis=1, keepdims=True) * (1 / m)

        dA1 = np.dot(w2.T, dZ2)
        dZ1 = dA1 * sigmoid(Z1) * (1 - sigmoid(Z1))
        dW1 = np.dot(dZ1, X_train.T) / m #######
        db1 = np.sum(dZ1, axis=1, keepdims=True) * (1 / m)

        w2 = w2 - learning_rate * dW2
        b2 = b2 - learning_rate * db2
        w1 = w1 - learning_rate * dW1
        b1 = b1 - learning_rate * db1

        if(i %1000 == 0):
            print(f'At epoch {i}, cost is {cost}')
    return [w1, w2, b1, b2]

In [244]:
def score_multi(X, y, w1, w2, b1, b2):
    # y is one-hot-encoded
    Z1 = np.dot(w1, X) + b1
    A1 = sigmoid(Z1) 
    Z2 = np.dot(w2, A1) + b2
    A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0) # SOFTMAX
    Y_predictions = np.argmax(A2, axis=0).reshape(750,1)
    labels = np.argmax(y, axis=0)
    print(classification_report(Y_predictions, labels))    

In [230]:
params = train_multi(X_train, yone_train,num_iter=200000, learning_rate=0.5, hidden_layer_num=18)

At epoch 0, cost is 2.5770290313751767
At epoch 1000, cost is 2.2981331255033393
At epoch 2000, cost is 2.264478985836349
At epoch 3000, cost is 2.0687115230972406
At epoch 4000, cost is 1.981307395982239
At epoch 5000, cost is 1.952453677723007
At epoch 6000, cost is 1.9350599606530732
At epoch 7000, cost is 1.9163748149140036
At epoch 8000, cost is 1.88847294962093
At epoch 9000, cost is 1.8478822796864753
At epoch 10000, cost is 1.80730034182107
At epoch 11000, cost is 1.7804664629468436
At epoch 12000, cost is 1.7652118646385995
At epoch 13000, cost is 1.7558475375026277
At epoch 14000, cost is 1.7491554140624597
At epoch 15000, cost is 1.7437060447091595
At epoch 16000, cost is 1.7388718861144334
At epoch 17000, cost is 1.7495292617374303
At epoch 18000, cost is 1.7431078211444122
At epoch 19000, cost is 1.7388163636005673
At epoch 20000, cost is 1.7345875479582737
At epoch 21000, cost is 1.7302088104386746
At epoch 22000, cost is 1.7255917944841495
At epoch 23000, cost is 1.72067

At epoch 190000, cost is 1.2982324391355775
At epoch 191000, cost is 1.3006413600028202
At epoch 192000, cost is 1.2821832708257663
At epoch 193000, cost is 1.302732859943975
At epoch 194000, cost is 1.293710018561376
At epoch 195000, cost is 1.2963933063608613
At epoch 196000, cost is 1.2904233830781822
At epoch 197000, cost is 1.2994188965754092
At epoch 198000, cost is 1.2928360580671254
At epoch 199000, cost is 1.3044683461656639


In [241]:
score_multi(X_test, yone_test, *params)

             precision    recall  f1-score   support

          0       0.71      0.41      0.52        37
          1       0.81      0.72      0.76        18
          2       0.47      0.40      0.43        35
          3       0.35      0.33      0.34        27
          4       0.00      0.00      0.00         0
          5       0.27      0.58      0.37        12
          6       0.85      0.74      0.79        31
          7       0.81      0.39      0.53        54
          8       0.45      0.45      0.45        20
          9       0.20      0.38      0.26        16

avg / total       0.61      0.47      0.51       250



/home/insiyeah/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [245]:
score_multi(X_train, yone_train, *params)

             precision    recall  f1-score   support

          0       0.62      0.42      0.50       118
          1       0.94      0.90      0.92        88
          2       0.51      0.43      0.47        84
          3       0.47      0.38      0.42        92
          4       0.04      1.00      0.08         3
          5       0.22      0.64      0.32        25
          6       0.73      0.69      0.71        77
          7       0.92      0.49      0.64       138
          8       0.53      0.64      0.58        66
          9       0.17      0.20      0.19        59

avg / total       0.63      0.52      0.56       750

